In [2]:
import pandas as pd
import fastparquet
import os

In [3]:
df = pd.read_parquet(
    "/home/usuario/Documentos/Escorpiao_series/Dados_processados/df1_recodificado.parquet",
    engine="fastparquet"
)

In [6]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180668 entries, 0 to 180667
Data columns (total 69 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   DT_SIN_PRI      180668 non-null  datetime64[ns]
 1   SEM_PRI         180668 non-null  object        
 2   ANO_NASC        180668 non-null  object        
 3   NU_IDADE_N      180668 non-null  int64         
 4   CS_SEXO         180583 non-null  object        
 5   CS_GESTANT      170048 non-null  object        
 6   CS_RACA         166897 non-null  object        
 7   CS_ESCOL_N      139586 non-null  object        
 8   ID_MN_RESI      180668 non-null  object        
 9   ID_OCUPA_N      180668 non-null  object        
 10  ANT_DT_ACI      180668 non-null  int64         
 11  ANT_UF          180668 non-null  object        
 12  ANT_MUNIC_      180668 non-null  object        
 13  ANT_LOCALI      180668 non-null  object        
 14  ANT_TEMPO_      170928 non-null  obj